<a href="https://colab.research.google.com/github/jrebull/ChileSistemasRecomendacion/blob/main/Rebull_4_practico_content_based_vDraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 Magister en Inteligencia Artificial

<center>

[![Institution](https://img.shields.io/badge/Institution-Pontificia%20Universidad%20Católica%20de%20Chile-003A70?style=for-the-badge&logo=graduation-cap)](https://www.uc.cl)
[![Course](https://img.shields.io/badge/Course-Sistemas%20de%20Recomendación-FF6B6B?style=for-the-badge&logo=star)](https://www.uc.cl)
[![Activity](https://img.shields.io/badge/Práctico%20Clase%204-Google%20Colab-F9AB00?style=for-the-badge&logo=googlecolab)](https://colab.research.google.com)

</center>
---

## 🎬 **Sistemas de Recomendación**

### 👨🏫 **Equipo Docente**
- **Profesor:** Vicente Domínguez
- **Profesor:** Michelle Anastasia Veroska Madrid Urrutia

---

## **<font color="#895cf9">Práctico Clase 4: Recomendación basada en Contenido (Texto)</font>**

### 📌 **Detalles de la Actividad**
- **Dataset:** Goodreads
- **Formato de entrega:** Notebook
- **Modalidad:** Individual
- **Fecha de entrega:** 16 de Noviembre 2025

---

## 👨🎓 **Alumno**
<center>
<img src="https://iili.io/KuvsGKx.png" alt="Javier Augusto Rebull Saucedo" width="150">

### **Javier Augusto Rebull Saucedo**
**Programa:** MNA - Tecnológico de Monterrey  
A01795838@tec.mx

📚 Estudiante Invitado
</center>

---

## **<font color="orange">Objetivo del Práctico</font>**

En este práctico implementaremos un sistema de recomendación de libros basado en contenido utilizando el dataset Goodreads:

- **Extracción de Embeddings**: Modelos BERT y BERT-large para representaciones vectoriales de textos
- **Reducción de Dimensionalidad**: Aplicación de PCA a 20 dimensiones
- **Cálculo de Similaridad**: Análisis de similitud entre items mediante métrica cosine
- **Evaluación mediante métricas de ranking**: MAP y NDCG@10

---

Utilizaremos modelos pre-entrenados de **Transformers** y **Sentence-BERT** para la generación de embeddings semánticos

## 📚 **Librerías Utilizadas**

| Librería | Función |
|----------|---------|
| **numpy** | Operaciones numéricas y manipulación de arrays multidimensionales |
| **json** | Lectura y escritura de archivos JSON (interacciones de usuarios) |
| **requests** | Descargas de datos desde URLs externas |
| **heapq** | Estructura de datos heap para ranking eficiente de recomendaciones |
| **math** | Funciones matemáticas básicas |
| **matplotlib** | Visualización de gráficos y resultados |
| **sklearn.metrics.pairwise_distances** | Cálculo de distancias entre vectores (cosine similarity) |
| **sklearn.decomposition.PCA** | Reducción de dimensionalidad de embeddings |
| **io.BytesIO** | Manejo de datos en memoria como streams binarios |
| **pickle** | Serialización y deserialización de objetos Python |
| **pandas** | Manipulación y análisis de datos tabulares (DataFrames) |
| **time** | Medición de tiempo de ejecución de algoritmos |

In [1]:
import numpy as np
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time

Descargamos datos que vienen previamente calculados:
- transacciones/interacciones de cada usuario
- transaciones para evaluar el modelo
- embeddings de descripciones calculados con BERT  
- embeddings de descripciones calculados con BERT-large
- datos de libros con información de titulo, descripcion, año de publicacion, entre otros.

In [2]:
%%capture
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

In [3]:
ls

books.csv                            goodreads_past_interactions.json
goodreads_bert_embeddings.npy        goodreads_test_interactions.json
goodreads_bert_large_embeddings.npy  sample_data/


## Todos los libros del catálogo de goodreads

In [10]:
df_books = pd.read_csv('books.csv', sep=',')
#df_books.head()
df_books

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,9993,4936457,4936457,5002120,27,224079948,9.780224e+12,Ian Mortimer,2008.0,The Time-Traveller's Guide to Medieval England...,...,12115,970,236,521,2497,4790,4071,https://images.gr-assets.com/books/1328167619m...,https://images.gr-assets.com/books/1328167619s...,Imagine you could get into a time machine and ...
4283,9994,4769651,4769651,4834466,2,810983559,9.780811e+12,"Michael Buckley, Peter Ferguson",2009.0,"The Everafter War (The Sisters Grimm, #7)",...,12965,455,110,289,1745,3989,6832,https://images.gr-assets.com/books/1388278230m...,https://images.gr-assets.com/books/1388278230s...,Picking up after the dramatic cliffhanger that...
4284,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,"The Edge lies between worlds, on the border be..."
4285,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...,"""O'Brian's Aubrey-Maturin volumes actually con..."


## Interacciones pasadas de los usuarios (ej. libros leídos)

In [5]:
# diccionario con id del usuario y id de libros con los que ha interactuado en el pasado
with open('goodreads_past_interactions.json') as f:
    user_interactions = json.load(f)

# diccionario con id del usuario y id de libros para testear el modelo
with open('goodreads_test_interactions.json') as f:
    user_interactions_test = json.load(f)


In [6]:
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

In [7]:
np.load('goodreads_bert_embeddings.npy').shape

(4287, 768)

In [8]:
np.load('goodreads_bert_large_embeddings.npy').shape

(4287, 1024)

# Cargar características pre-entrenadas: BERT y BERT-large

En esta sección se trabajará con modelos pre-entrenados de modelos de lenguage BERT y BERT-large que convierten texto a embeddings.

Bidirectional Encoder Representations from Transformers (BERT) es una técnica de NLP (Natural Language Processing) desarrollada por Google y publicada en 2018 por Jacob Devlin.

Actualmente Google utiliza BERT para entender las consultas de los usuarios en su buscador.

Tiene dos versiones:
- **BERT:** 12 capas, 12 cabezales de atencion y 110 millones de parámetros. Genera vectores de 768 dimensiones
- **BERT-large:** 24 capas, 16 cabezales de atencion y 340 millones de parámetros.  

![BERT y BERT-large](http://jalammar.github.io/images/bert-base-bert-large.png)

![BERT y BERT-large arquitectura](http://jalammar.github.io/images/bert-base-bert-large-encoders.png)

En este caso los textos que utilizaremos son los títulos de los libros con su descripción y compararemos los resultados de recomendación con BERT y BERT-large. Para efectos de este trabajo los vectores de características ya fueron entrenados y guardados en archivos numpy. A continuación son cargados en memoria.

Para mayores detalles sobre el modelo de lenguaje BERT se recomienda revisar el siguiente artículo:
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)


In [11]:
# cargamos los embeddings de BERT y de BERT-large
bert_featmat = np.load('goodreads_bert_embeddings.npy', allow_pickle=True)
bert_large_featmat = np.load('goodreads_bert_large_embeddings.npy', allow_pickle=True)

In [12]:
bert_featmat.shape # BERT son vectores de 768 dimensiones para cada libro (4287)

(4287, 768)

In [13]:
bert_large_featmat.shape # BERT-large son vectores de 1024 dimensiones para cada libro (4287)

(4287, 1024)

### **Pregunta 1 (2 ptos)**

Considerando que haremos un recomendador basado en contenido ¿Por qué el uso de modelos de lenguage es una buena elección para este tipo de problema?

### **<font color="\#895cf9">Respuesta 1:</font>**

> En el contexto de este práctico, estamos construyendo un **sistema de recomendación de libros basado en contenido**. El "contenido" fundamental que define cada libro es su **título y descripción textual**. El desafío es: ¿cómo podemos comparar miles de descripciones de libros de forma significativa para encontrar similitudes?
>
> Aquí es donde los modelos de lenguaje (LMs) como **BERT** se convierten en la herramienta ideal. A diferencia de métodos más simples (como contar palabras clave), estos modelos están diseñados para **comprender el significado semántico y el contexto** del lenguaje.

Las **ventajas clave** de usar modelos de lenguaje para *este* problema son:

  * **Comprensión Semántica Profunda:** Un modelo como BERT no solo ve palabras, sino que "entiende" conceptos. Por ejemplo, puede reconocer que "una novela distópica para jóvenes adultos" (como *The Hunger Games*) y "un romance paranormal" (como *Twilight*) son géneros muy diferentes, incluso si ambos usan palabras como "peligro" o "amor". Captura la **esencia** de la trama.

  * **Conversión a Embeddings:** La principal ventaja práctica es que BERT transforma el texto de cada libro en un **embedding**: un vector numérico de alta dimensionalidad (768 para BERT-base y 1024 para BERT-large). Este vector es una "huella digital" matemática del significado del libro.

  * **Similitud Cuantificable:** Una vez que todos los libros son vectores, podemos usar métricas matemáticas (como la **similitud coseno**, que se usa en la función `recommend` de este notebook) para calcular la "distancia" entre ellos. Los libros con vectores "cercanos" son semánticamente similares, y por lo tanto, excelentes recomendaciones para alguien que leyó uno de ellos.

  * **Superación del Problema de "Arranque en Frío" (Cold Start):** Este enfoque funciona excelentemente para libros nuevos. Tan pronto como un nuevo libro se añade al catálogo con su descripción, podemos generar su embedding y recomendarlo de inmediato, sin necesidad de esperar a que los usuarios interactúen con él.

A continuación, se presenta una tabla resumen con las ventajas y desventajas de este enfoque en el contexto del práctico:

| Ventajas | Desventajas |
| :--- | :--- |
| <font color="#00FF41">**Comprensión Semántica Profunda**</font> | <font color="#FF6B6B">**Alto Costo Computacional**</font> |
| Entiende el *significado* y contexto de la trama, no solo palabras clave. | Generar los embeddings (los archivos `.npy`) es un proceso lento que requiere hardware potente (como GPUs). |
| <font color="#00FF41">**Calidad del Embedding**</font> | <font color="#FF6B6B">**Alta Dimensionalidad**</font> |
| Crea vectores densos que capturan la "esencia" y matices del libro. | Los vectores (768 o 1024 dim.) hacen que el cálculo de similitud sea lento, un problema que se aborda con PCA. |
| <font color="#00FF41">**Soluciona el "Cold Start"**</font> | <font color="#FF6B6B">**Dependencia de la Calidad del Contenido**</font> |
| Un libro nuevo puede ser recomendado al instante solo con su descripción. | Si la descripción de un libro es mala, corta o engañosa, las recomendaciones serán de baja calidad (*Garbage In, Garbage Out*). |
| <font color="#00FF41">**Flexibilidad de Lenguaje**</font> | <font color="#FF6B6B">**Falta de Interpretabilidad**</font> |
| Maneja sinónimos y frases complejas (ej. "suspenso" vs. "thriller"). | Es difícil saber *exactamente por qué* el modelo considera que dos libros son semánticamente similares (es una "caja negra"). |

-----

**🔗 Referencia:**

  * Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). **BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding**. *arXiv preprint arXiv:1810.04805*. [https://arxiv.org/abs/1810.04805](https://arxiv.org/abs/1810.04805)

# Probamos con BERT y BERT-large reduciendo dimensionalidad con PCA a 20 dimensiones.

Una vez calculado (o cargado) los vectores característicos de cada libro a partir de su titulo y descripción, reducimos dimensionalidad. Probaremos con BERT y BERT-large para comparar los resultados de ambos en recomendación basada en contenido.


In [14]:
# Project into a 20 PCA feature space
pca20_bert_featmat = PCA(n_components=20).fit_transform(bert_featmat)
pca20_bert_large_featmat = PCA(n_components=20).fit_transform(bert_large_featmat)

In [15]:
pca20_bert_featmat.shape

(4287, 20)

In [16]:
pca20_bert_large_featmat.shape

(4287, 20)

### **Pregunta 2 (2 ptos)**

Comente por qué si se utiliza PCA para reducir la dimensión de cada vector a 20 la velocidad (tiempo) de hacer recomendación puede disminuir?

### **<font color="#895cf9">Respuesta 2:</font>**

> El uso de PCA no *disminuye la velocidad* (haciéndola más lenta), sino que *disminuye el **tiempo** de ejecución* (haciéndola **mucho más rápida**). Los resultados observados en este notebook lo confirman.

La razón de esta ganancia de velocidad es que atacamos directamente el **cuello de botella computacional** de nuestro recomendador: el cálculo de la similitud coseno.

Este fenómeno se conoce como la **"Maldición de la Dimensionalidad"**. En resumen: **menos números = menos matemáticas**.

-----

### El Problema: El Costo de `pairwise_distances`

El corazón de nuestra función `recommend` es la línea que calcula `pairwise_distances` usando la métrica `'cosine'`. Esta función tiene que:

1.  Tomar el vector de un libro que le gustó al usuario.
2.  Comparar ese vector contra los **4287 vectores** de todos los libros del catálogo.
3.  Repetir esto para *cada libro* que el usuario ha leído.

El cálculo de la similitud coseno, aunque eficiente, implica operaciones matemáticas (productos punto, sumas, raíces cuadradas) que **escalan linealmente con la dimensionalidad ($d$)** del vector.

### ⚡ La Solución: Reducir $d$ con PCA

Al usar PCA, no estamos cambiando la *lógica* del recomendador, sino la *materia prima* con la que trabaja.

  * **Sin PCA (BERT-large):** El `embedding` tiene un shape de `(4287, 1024)`. Cada cálculo de similitud debe procesar vectores de **1024 dimensiones**.
  * **Con PCA (pca20\_bert\_large\_featmat):** El `embedding` tiene un shape de `(4287, 20)`. Cada cálculo de similitud procesa vectores de solo **20 dimensiones**.

Estamos, en efecto, reduciendo la cantidad de operaciones matemáticas en un factor de $\approx 51$ (es decir, $1024 / 20 \approx 51$).

### La Evidencia (Nuestros Resultados de este Notebook)

Como se ve en las celdas de evaluación de este notebook, el impacto en el tiempo es drástico:

| Modelo | Dimensiones ($d$) | Tiempo de Ejecución Total | Ganancia de Velocidad |
| :--- | :--- | :--- | :--- |
| BERT | 768 | 356.87 segs | - |
| **BERT con PCA-20** | **20** | **19.22 segs** | **¡\~18.5x más rápido\!** |
| BERT-large | 1024 | 550.93 segs | - |
| **BERT-large con PCA-20** | **20** | **17.33 segs** | **¡\~31.8x más rápido\!** |

**Conclusión:** Al reducir drásticamente la dimensionalidad (de 1024 a 20), reducimos la cantidad de cálculos necesarios para encontrar los vecinos más cercanos, lo que **disminuye brutalmente el tiempo de ejecución** y hace que el recomendador sea mucho más rápido.

-----

**🔗 Referencia:**

Este concepto de usar descomposición de matrices (como PCA o SVD) para reducir la dimensionalidad y acelerar la búsqueda semántica es la base de una de las técnicas más influyentes en la recuperación de información:

  * Deerwester, S., Dumais, S. T., Furnas, G. W., Landauer, T. K., & Harshman, R. (1990). **Indexing by latent semantic analysis**. *Journal of the American Society for Information Science*, 41(6), 391-407.
  [http://wordvec.colorado.edu/papers/Deerwester_1990.pdf](http://wordvec.colorado.edu/papers/Deerwester_1990.pdf)

# Recomendaciones

In [17]:
# format results
pd.options.display.max_colwidth = 50
#pd.set_option('display.max_colwidth', -1)

In [18]:
def recommend(embedding, user_id=None, topk=10, metric='cosine'):

    user_id = str(user_id)

    #Calculate distance metrics
    trx = user_interactions[user_id]
    n = embedding.shape[0]
    distances = 1e9

    # recorremos transacciones pasadas del usuario
    for t in trx:
        query_i = bookid2idx[t]

        # recomendamos items más cercanos a items con los que interactuó el usuario
        distances = np.minimum(distances, pairwise_distances(
                embedding[query_i].reshape(1,-1), embedding, metric=metric).reshape(-1))

    #Rank items de menor a mayor distancia (nos quedamos con los topk)
    trx_set = set(trx)
    heap = []

    for i in range(n):
        if idx2bookid[i] in trx_set:
            continue
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[i], i))
        else:
            heapq.heappushpop(heap, (-distances[i], i))

    heap.sort(reverse=True)

    # utilizamos un heap para extraer los items ordenados de menor a mayor distancia
    recommended_ids = [idx2bookid[i] for _,i in heap]

    # retornar los que el usuario no haya consumido
    filtered_recommended_ids = []

    return recommended_ids

## generar recomendaciones para un usuario en particular

In [19]:
import random

random.sample(sorted(user_interactions.keys()), 10) # vemos 10 usuarios aleatorios

['20430',
 '44034',
 '40755',
 '12697',
 '4642',
 '3480',
 '27758',
 '7995',
 '920',
 '46632']

In [20]:
# recomendación para el usuario id = 49299 , utilizando bert con reduccion de dimensionalidad a 20
user_id = '32204'
rec = recommend(pca20_bert_featmat, user_id=user_id, topk=15)
rec

[9785,
 6265,
 7032,
 10,
 6250,
 3349,
 6865,
 7701,
 9941,
 5244,
 48,
 5126,
 2847,
 390,
 2796]

## transacciones pasadas del usuario

In [21]:
past_interactions = user_interactions[str(user_id)]

df_books[df_books['book_id'].isin(past_interactions)][['book_id', 'original_title']]

,book_id,original_title
22,26,The Da Vinci Code
50,61,The Girl on the Train
91,114,Tuesdays with Morrie
96,119,The Handmaid's Tale
99,123,The Firm
...,...,...
2610,4961,The Fifth Season
2615,4972,Straight Man
2849,5578,The Nix
4138,9504,The Obelisk Gate


## información de recomendaciones

In [22]:
df_books[df_books['book_id'].isin(rec)][['book_id', 'original_title']]

,book_id,original_title
8,10,Pride and Prejudice
39,48,Fahrenheit 451
293,390,The Strange Case of Dr Jekyll and Mr Hyde
1643,2796,The Lost Wife
1673,2847,La Vérité sur l'affaire Harry Quebert
1921,3349,Se questo è un uomo
2676,5126,Travesuras de la niña mala
2718,5244,Il barone rampante
3105,6250,Soumission
3111,6265,L’écume des jours


# Evaluación de las recomendaciones

In [23]:
# Métricas de evaluación
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg


## Evaluación de recomendación con BERT

In [24]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [25]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))


MAP  0.03669047619047619
ndcg@10 0.13
tiempo de ejecucion 356.87 segs


## Evaluación de recomendación con BERT reducidos con PCA-20

In [26]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(pca20_bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [27]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.00453968253968254
ndcg@10 0.03
tiempo de ejecucion 19.22 segs


## Evaluación de recomendación con BERT-large

In [28]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [29]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.060920634920634924
ndcg@10 0.24
tiempo de ejecucion 550.93 segs


## Evaluación de recomendación con BERT-large reducidos con PCA-20

In [30]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(pca20_bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [31]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.026123015873015874
ndcg@10 0.07
tiempo de ejecucion 17.33 segs


### Pregunta 3 (4 ptos):
- Comente los resultados en términos de tiempo de ejecución y métricas de ranking para los 4 modelos. (BERT, BERT-large, BERT reducido con PCA a 20 dimensiones , BERT-large reducido con PCA a 20 dimensiones)


### **<font color="#895cf9">Respuesta 3:</font>**

Al analizar los 4 modelos, mis resultados muestran un **trade-off clásico y muy claro entre la calidad de la recomendación (métricas de ranking) y la eficiencia (tiempo de ejecución)**.

Para que se entienda mejor, aquí están los datos que obtuve:

| Modelo | MAP | NDCG@10 | Tiempo de Ejecución |
| :--- | :---: | :---: | :---: |
| BERT (768-dim) | 0.0367 | 0.13 | 356.87 seg |
| BERT-large (1024-dim) | **0.0609** | **0.24** | 550.93 seg |
| BERT + PCA-20 | 0.0045 | 0.03 | 19.22 seg |
| BERT-large + PCA-20 | 0.0261 | 0.07 | **17.33 seg** |

-----

### Análisis de Calidad (Métricas de Ranking)

En cuanto a la calidad de la recomendación, mi conclusión es: **"más grande es mejor"**.

  * **`BERT-large` (full) es el ganador indiscutible.** Con un MAP de $0.061$ y un NDCG de $0.24$, barre por completo al `BERT` base (MAP $0.037$, NDCG $0.13$). Esto demuestra que los embeddings de 1024 dimensiones capturan el significado semántico de las descripciones de los libros con mucho más detalle y precisión que los de 768.
  * **La compresión (PCA) destruye la calidad.** Aquí no hay sorpresas. Al comprimir 1024 dimensiones a solo 20, estamos perdiendo una cantidad masiva de información semántica. Esto se ve en el colapso de las métricas para ambos modelos (ej. el MAP de `BERT-large` cae de $0.061$ a $0.026$). El modelo `BERT` base + PCA es el peor de todos, con un MAP de $0.0045$ que es casi inútil.
  * **Interesante:** Incluso después de la compresión, `BERT-large + PCA-20` (MAP $0.026$) sigue siendo mucho mejor que `BERT + PCA-20` (MAP $0.0045$). Esto me dice que el embedding inicial de `BERT-large` era tan superior, que incluso después de "exprimirlo" retuvo más información valiosa que el `BERT` base.

-----

### Análisis de Eficiencia (Tiempo de Ejecución)

Aquí es donde PCA se roba el show y justifica su existencia.

  * **La velocidad de PCA es brutal.** Pasamos de **550 segundos** con `BERT-large` a solo **17 segundos** con `BERT-large + PCA-20`. ¡Es **\~32 veces más rápido**\! Lo mismo para `BERT` base, que pasó de **357 segundos** a **19 segundos** (un speedup de \~18.5x).
  * **La dimensionalidad es el cuello de botella.** Los dos modelos con PCA (ambos de 20 dimensiones) tienen tiempos casi idénticos (17.33s vs 19.22s). Esto confirma exactamente lo que vimos en la Pregunta 2: el tiempo de ejecución depende casi por completo del cálculo de similitud coseno, y este a su vez, de la dimensionalidad. Al tener ambos la misma dimensión (20), sus tiempos de ejecución son prácticamente iguales.

-----

### Veredicto Final

Después de ver estos números, mi elección depende totalmente del caso de uso:

1.  **Para Máxima Calidad (ej. un trabajo offline):** Usaría **`BERT-large` (full)** sin dudarlo. Sus métricas de ranking son las mejores por mucho, y si no tengo un usuario esperando una respuesta en tiempo real, el costo de 9 minutos (550 segs) es aceptable.
2.  **Para un Sistema en Producción (ej. recomendación en vivo):** El ganador práctico es **`BERT-large + PCA-20`**. Es increíblemente rápido (17 segundos para todo el test set) y, aunque sus métricas bajan, su MAP de $0.026$ es **casi tan bueno como el del `BERT` base completo** (MAP $0.037$), pero lográndolo en una fracción minúscula del tiempo. Ofrece el mejor equilibrio entre velocidad y calidad.

---

## Conclusiones

Tras el desarrollo y la evaluación de los cuatro modelos de recomendación basada en contenido, se pueden extraer las siguientes conclusiones:

* **La Complejidad del Modelo Importa:** El modelo **`BERT-large`** (1024 dimensiones) obtuvo los mejores resultados de *calidad* (MAP: $0.061$, NDCG@10: $0.24$), demostrando que su embedding de mayor dimensionalidad captura de manera más rica y precisa el significado semántico de las descripciones de los libros.
* **La Eficiencia es el Cuello de Botella:** La reducción de dimensionalidad con **PCA a 20 dimensiones** probó ser una estrategia de optimización de velocidad extremadamente efectiva. Logró reducir el tiempo de ejecución del modelo `BERT-large` en un factor de **~32x** (de 550 a 17 segundos), confirmando que el cálculo de similitud en altas dimensiones es el principal cuello de botella.
* **Existe un Claro Trade-off:** La ganancia en velocidad de PCA tiene un costo directo en la calidad de la recomendación. Al comprimir la información semántica de 1024 a 20 dimensiones, las métricas de `BERT-large` se vieron significativamente afectadas (el MAP cayó de $0.061$ a $0.026$).
* **El Equilibrio es la Clave:** Para un escenario de producción donde la velocidad de respuesta es crítica, el modelo **`BERT-large + PCA-20`** emergió como la solución de *mejor equilibrio*. A pesar de su compresión, sus métricas (MAP $0.026$) fueron casi tan buenas como las del modelo `BERT-base` *completo* (MAP $0.037$), pero operando **~18.5 veces más rápido**.

---

## Reflexión Personal

Este práctico ha sido una excelente demostración de cómo conectar dos mundos que a menudo se estudian por separado: **Procesamiento de Lenguaje Natural (NLP)** y **Sistemas de Recomendación**.

Ver la teoría de la **"Maldición de la Dimensionalidad"** en acción fue lo más revelador. Es un concepto que uno lee en los libros, pero ver cómo el tiempo de ejecución colapsa de **9 minutos (550s) a solo 17 segundos** al reducir los vectores de 1024 a 20 dimensiones, lo hace tangible. El costo es real.

Al mismo tiempo, me forzó a pensar críticamente sobre las métricas. Un modelo `BERT-large + PCA-20` puede parecer "peor" en papel que el `BERT-base` completo, pero en la práctica, ofrece una experiencia de usuario (velocidad) dramáticamente superior por una pérdida de precisión casi imperceptible para el usuario final.

Este ejercicio me deja reflexionando sobre técnicas de reducción de dimensionalidad más avanzadas (como Autoencoders o UMAP) que quizás podrían preservar mejor la información semántica en espacios de baja dimensión, logrando lo mejor de ambos mundos: velocidad y precisión.

---

## Referencias

* Domínguez, V. (2025, Noviembre). *Clase N4 - RecSys MIA: Recomendación basada en contenido (text mining, nlp, aplicación)* \[Diapositivas de clase]. Magister en Inteligencia Artificial, Pontificia Universidad Católica de Chile.
* Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). *BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding*. arXiv preprint arXiv:1810.04805.
* Deerwester, S., Dumais, S. T., Furnas, G. W., Landauer, T. K., & Harshman, R. (1990). *Indexing by latent semantic analysis*. Journal of the American Society for Information Science, 41(6), 391-407.